**SPEED LIMIT MODEL**

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import matplotlib.pyplot as plt

dataset_path = "speed_limit_dataset_final.csv"
df = pd.read_csv(dataset_path)

X = df[['acceleration', 'distance_km', 'battery_level', 'terrain_elevation', 'weather_condition']]
y = df['speed_limit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.3,
    objective='reg:squarederror',
    random_state=42
)

model.fit(X_train, y_train)

cv_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=5)
print(f"Cross-Validation MAE: {-cv_scores.mean():.2f}")

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")

# plt.figure(figsize=(10, 6))
# plt.scatter(range(len(y_test)), y_test, label="Actual Speed Limit", alpha=0.6)
# plt.scatter(range(len(y_pred)), y_pred, label="Predicted Speed Limit", alpha=0.6)
# plt.xlabel("Test Sample Index")
# plt.ylabel("Speed Limit (km/h)")
# plt.title("Actual vs Predicted Speed Limit")
# plt.legend()
# plt.show()

model_path = "speed_limit.pkl"
joblib.dump(model, model_path)
print(f"Model saved as {model_path}")


Cross-Validation MAE: 0.95
Mean Absolute Error: 0.95
Root Mean Squared Error: 1.46
Model saved as speed_limit.pkl


In [3]:
print("Enter values for prediction:")
acceleration = float(input("Acceleration (m/s²): "))
distance = float(input("Distance (km): "))
battery_level = float(input("Battery Level (%): "))
terrain = int(input("Terrain Elevation (-2 to +2): "))
weather = int(input("Weather Condition (0=Normal, 1=Hot, 2=Wet): "))

user_input = np.array([[acceleration, distance, battery_level, terrain, weather]])
user_prediction = model.predict(user_input)
print(f"Predicted Speed Limit: {user_prediction[0]:.2f} km/h")

Enter values for prediction:
Acceleration (m/s²): 1.6
Distance (km): 60
Battery Level (%): 42
Terrain Elevation (-2 to +2): 1
Weather Condition (0=Normal, 1=Hot, 2=Wet): 0
Predicted Speed Limit: 83.38 km/h


In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

comparison_df = pd.DataFrame({'Actual Speed Limit': y_test, 'Predicted Speed Limit': y_pred})
print("\nActual vs Predicted Speed Limits:")
print(comparison_df.head(10))


Mean Absolute Error (MAE): 0.95
Root Mean Squared Error (RMSE): 1.46
R² Score: 1.00

Actual vs Predicted Speed Limits:
      Actual Speed Limit  Predicted Speed Limit
1501          115.000000             115.651588
2586           80.370982              81.214783
2653          120.000000             121.327797
1055           84.399779              85.215004
705           105.670593             107.063057
106            71.597672              70.294350
589            68.288075              66.551903
2468          101.753956             103.030006
2413          120.000000             120.173416
1600           90.360005              94.409584


In [5]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, test_pred)

print(f"Train R² Score: {train_r2:.2f}")
print(f"Test R² Score: {test_r2:.2f}")

Train R² Score: 1.00
Test R² Score: 1.00


**WEAR AND TEAR MODEL**

In [11]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

dataset_path = "wear_tear_dataset.csv"
df = pd.read_csv(dataset_path)

x_labels = [
    "braking_intensity",
    "harsh_acceleration",
    "average_speed",
    "distance_driven",
    "weather_condition",
    "terrain_type"
]

y_labels = [
    "battery_health",
    "brake_health",
    "tyre_health"
]

X = df[x_labels]
y = df[y_labels]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

optimized_model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.03,
    max_depth=7,
    subsample=0.85,
    colsample_bytree=0.9,
    reg_alpha=0.05,
    reg_lambda=0.2,
    gamma=0.3,
    objective='reg:squarederror',
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Performance:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² Score: {r2:.2f}")

model_filename = "wear_tear.pkl"
joblib.dump(model, model_filename)
print(f"\nModel saved as '{model_filename}'")

Model Performance:
Mean Absolute Error (MAE): 1.77
Mean Squared Error (MSE): 4.87
R² Score: 0.97

Model saved as 'wear_tear.pkl'


In [12]:
test_input = pd.DataFrame({
    "braking_intensity": [np.random.uniform(0, 10)],
    "harsh_acceleration": [np.random.uniform(0, 10)],
    "average_speed": [np.random.uniform(20, 120)],
    "distance_driven": [np.random.uniform(5, 150)],
    "weather_condition": [np.random.choice([0, 1, 2])],
    "terrain_type": [np.random.choice([-2, -1, 0, 1, 2])]
})

predicted_health = model.predict(test_input)

print("\nRandom Test Input:")
print(test_input)

print("\nPredicted Wear & Tear Health:")
print(f"Battery Health: {predicted_health[0][0]:.2f}%")
print(f"Brake Health:   {predicted_health[0][1]:.2f}%")
print(f"Tyre Health:    {predicted_health[0][2]:.2f}%")



Random Test Input:
   braking_intensity  harsh_acceleration  average_speed  distance_driven  \
0           9.511929            9.931051      72.354778        26.098173   

   weather_condition  terrain_type  
0                  0             2  

Predicted Wear & Tear Health:
Battery Health: 45.43%
Brake Health:   46.76%
Tyre Health:    55.44%


In [15]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_test = np.ravel(y_test)
y_pred = np.ravel(y_pred)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

comparison_df = pd.DataFrame({
    'Battery Health': y_test,
    'Predicted Battery Health': y_pred
})

print("\nActual vs Predicted:")
print(comparison_df.head(10))

Mean Absolute Error (MAE): 1.77
Root Mean Squared Error (RMSE): 2.21
R² Score: 0.98

Actual vs Predicted:
   Battery Health  Predicted Battery Health
0       52.224825                 54.251976
1       47.492306                 51.390366
2       65.085547                 68.056602
3       43.175191                 46.719452
4       55.799759                 55.172634
5       64.688139                 62.439911
6       67.075370                 67.463943
7       79.737283                 72.975960
8       80.095509                 79.046112
9       34.812648                 37.477436


In [17]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, y_pred)

print(f"Train R² Score: {train_r2:.2f}")
print(f"Test R² Score: {test_r2:.2f}")

Train R² Score: 0.99
Test R² Score: 0.98
